# salsbury

## Load Libraries

In [56]:
# Selenium
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# Reqeusts
import requests
# Other tools
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
from datetime import date
 
import csv
import pandas as pd
import time
import json
import fnmatch
import os
import tabula
from tabula.io import read_pdf


In [85]:
start_date = datetime.strptime("2020-12-12", '%Y-%m-%d')
start_date = start_date + timedelta(days=2)

start_date = (start_date.date())
print(start_date)
# start_date  < date.today()
print(start_date)
print('---')

2020-12-14
2020-12-14
---


['2017-07-02',
 '2017-10-30',
 '2018-02-27',
 '2018-06-27',
 '2018-10-25',
 '2019-02-22',
 '2019-06-22',
 '2019-10-20',
 '2020-02-17',
 '2020-06-16',
 '2020-10-14',
 '2021-02-11',
 '2021-06-11',
 '2021-10-07']

In [ ]:
# get to the page, insert each element from our list to our searchbox and then download the html that it returns 



## Functions Using Requests

In [2]:
def requests_get_item(url, item):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    if item == "html":       
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.text, 'html.parser')
        return(soup)
    
    elif item == "pdf":
        page = requests.get(url, headers = headers, stream=True)
        return(page)
    else:
        print("Valid Item Not Selected")

def download_pdfs(ls_pdf_urls, download_path, file_name):
    counter = 0 
    for pdf_url in ls_pdf_urls:
        counter = counter +1
        g = requests_get_item(pdf_url, "pdf")
        with open(f'{download_path}{file_name}_{counter}.pdf', 'wb') as sav:
            for chunk in g.iter_content(chunk_size=1000000):
                sav.write(chunk)
        print(f"download number: {counter}")
                
                
def convert_pdf_to_csv(pdf_directory, csv_directory):
    directory = fr'{pdf_directory}'
    directory_output = fr'{csv_directory}'
    count = 0
    for file in os.listdir(directory):        
        if file.endswith(".pdf"):
            count = count + 1 
            print(f'{directory}{file}: Conversion {count}')
            tabula.convert_into(f'{directory}{file}', f'{directory_output}{count}.csv', output_format="csv", pages='all')
    
    
    
    

## Functions Using Selenium

In [ ]:
# Define important paths
executable_path = "C:/Users/nicho/Documents/Code/university_police_logs/webscrapers/chromedriver.exe"
download_path = f"C:\\Users\\nicho\\Documents\\Code\\university_police_logs\\data\\handmade\\gt\\pdfs\\"

# Define Routes (usually in xpaths)

def browser_instance(executable_path, headless, download_path):
    
    
    chrome_options =  Options()
    if headless:
        chrome_options.add_argument("--headless")   
    chrome_options.add_experimental_option("prefs", {
            "download.default_directory": download_path})    
    driver = webdriver.Chrome(executable_path=executable_path, options = chrome_options)
    return driver  

def signIn(driver, url, signin_field, username_field, password_field):  
    driver.get(url)
    sign_in = driver.find_element_by_xpath(signin_field)
    sign_in.click()
    email_field = driver.find_element_by_xpath(username_field)
    password_field = driver.find_element_by_xpath(password_field)
    email_field.send_keys(USERNAME)
    password_field.send_keys(PASSWORD + Keys.RETURN)


## Useful Selenium Concepts
### Waiting until element is clickable
button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, download_all_field)))
button.click()

### Try and Expect
try:
    attempt to do something
except:
    do this if the try doesn't work out
    


## General Functions

In [ ]:
def write_json_data(data_out, database_name, file_path):
        #writing data to json
        data_out = data_out
        database_name = database_name
        file = f'{file_path}/{database_name}'
        with open(file, 'w') as outfile:
            json.dump(data_out, outfile)
            
def open_json_data(file_path):    
    with open(file_path) as f:
        data = json.load(f)
    return data

## Get 'em links

In [44]:
#script

page = requests_get_item('https://www.frostburg.edu/about-frostburg/Administrative-Offices/university-police/police-crime-log.php' , 'html')
div = page.find('div', class_="main-content")
# print(div)
link_list = []
final_link_list = []
base_url = 'https://www.frostburg.edu/about-frostburg/Administrative-Offices/university-police/'

links = div.find_all('a')

for link in links:
    if 'href' in str(link):
        link = (link["href"])
        link_list.append(link)
        print(link)
print(final_link_list)


_files/pdfs/logs/2021-crime-log.pdf
_files/pdfs/logs/2011-20xx-police-crime-log.pdf
[]


In [45]:

final_link_list = []
base_url = 'https://www.frostburg.edu/about-frostburg/Administrative-Offices/university-police/'
for link in link_list:
#     print(link)
#     print('---')
    if link.startswith("_files"):
        final_link = base_url + link
        final_link_list.append(final_link)
        print(final_link_list)
        print('---')
    else:
        print('check url format')

['https://www.frostburg.edu/about-frostburg/Administrative-Offices/university-police/_files/pdfs/logs/2021-crime-log.pdf']
---
['https://www.frostburg.edu/about-frostburg/Administrative-Offices/university-police/_files/pdfs/logs/2021-crime-log.pdf', 'https://www.frostburg.edu/about-frostburg/Administrative-Offices/university-police/_files/pdfs/logs/2011-20xx-police-crime-log.pdf']
---


## Download 'em

In [48]:
download_path = "../data/handmade/frostburg/pdfs/"
download_pdfs(final_link_list, download_path, "towson")

download number: 1
download number: 2


## Convert 'em

In [49]:
csv_directory = "../data/handmade/frostburg/csvs/"
convert_pdf_to_csv(download_path, csv_directory)

../data/handmade/frostburg/pdfs/towson_2.pdf: Conversion 1
../data/handmade/frostburg/pdfs/towson_1.pdf: Conversion 2
